# Ingest Data into GCP 
## Local Development Notebook

In [5]:
from utils import dict_from_yaml
from ingest import IngestYahoo

### Check out Api

### Run the Process Locally

In [6]:
config = dict_from_yaml('config.yaml')
ingest_yahoo = IngestYahoo(config)

### Check Config & Method Help

In [7]:
ingest_yahoo.config

{'env': 'dev',
 'api': {'name': 'yfinance',
  'tables': {'stock_history': {'tickers': ['MSFT', 'GOOGL', 'AAPL'],
    'period': '5d'}}},
 'bigquery': {'dest_dataset_id': 'yfinance_raw',
  'key_file': 'data-science-on-gcp-323609-ec18149ed324.json'}}

In [ ]:
help(ingest_yahoo.run)

Extracting the data creates a dictionary of dataframes with the raw format that is given by the api.

In [8]:
df_dict_raw = ingest_yahoo.extract()
df_dict_raw['stock_history'].head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker
Date,,,,,,,,
2023-04-10 00:00:00-04:00,289.209991,289.600006,284.709991,289.390015,23103000,0.0,0.0,MSFT
2023-04-11 00:00:00-04:00,285.750000,285.980011,281.640015,282.829987,27276600,0.0,0.0,MSFT
2023-04-12 00:00:00-04:00,284.790009,287.010010,281.959991,283.489990,27403400,0.0,0.0,MSFT
2023-04-13 00:00:00-04:00,283.589996,289.899994,283.170013,289.839996,24222700,0.0,0.0,MSFT
2023-04-14 00:00:00-04:00,287.000000,288.480011,283.690002,286.140015,20973900,0.0,0.0,MSFT


In the transformation step, we reset the index to be ready for upload to GCP.

In [9]:
df_dict_transformed = ingest_yahoo.transform(df_dict_raw)
df_dict_transformed['stock_history'].head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker
0,2023-04-10 00:00:00-04:00,289.209991,289.600006,284.709991,289.390015,23103000,0.0,0.0,MSFT
1,2023-04-11 00:00:00-04:00,285.750000,285.980011,281.640015,282.829987,27276600,0.0,0.0,MSFT
2,2023-04-12 00:00:00-04:00,284.790009,287.010010,281.959991,283.489990,27403400,0.0,0.0,MSFT
3,2023-04-13 00:00:00-04:00,283.589996,289.899994,283.170013,289.839996,24222700,0.0,0.0,MSFT
4,2023-04-14 00:00:00-04:00,287.000000,288.480011,283.690002,286.140015,20973900,0.0,0.0,MSFT


## Test Run Method

You must run the first and second setup steps first.

In [4]:
ingest_yahoo.run(env='dev')

Extracting data from endpoint: yfinance
Loaded 15 rows and 9 columns to data-science-on-gcp-323609.yfinance_raw.stock_history
